# 1. Preprocessing

In [ ]:
"""
Populates preprocessing/proteomes folder with the following files:
    - tsv file with protein IDs and their corresponding sequences for your selected species
    - json file with protein IDs as keys and sequences as values for your selected species
"""

import os, json
from preprocessing.Preprocessing import Preprocessing

preprocessing = Preprocessing()

# ------------ Modifiable ---------
taxon_ID = 243273 # set taxon ID for the species of interest: for example, 243273 is mycoplasma, 9606 is human, 10090 is mouse
# ---------------------------------

# check if files already exist
tsv_file = f'preprocessing/proteomes/protein-list-{taxon_ID}.tsv'
json_file = f'preprocessing/proteomes/protein-data-{taxon_ID}.json'

print(f"Checking for existing files for taxon ID: {taxon_ID}")
print(f"TSV file exists: {os.path.exists(tsv_file)}")
print(f"JSON file exists: {os.path.exists(json_file)}")

# only run preprocessing if files don't exist
if not os.path.exists(tsv_file) or not os.path.exists(json_file):
    print(f"\nRunning preprocessing for taxon ID: {taxon_ID}")
    preprocessing.get_proteome(taxon_ID)
    preprocessing.tsv_to_json(taxon_ID)
    preprocessing.check_entries(taxon_ID)
else:
    print(f"\nFiles already exist. Skipping preprocessing.")

# verify that files were created successfully
print(f"\nVerification:")
print(f"TSV file exists: {os.path.exists(tsv_file)}")
print(f"JSON file exists: {os.path.exists(json_file)}")

if os.path.exists(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
        print(f"Number of proteins processed: {len(data)}")
        if data:
            # show first protein entry as an example
            first_protein = list(data.keys())[0]
            print(f"Example first protein {first_protein}: {data[first_protein]['sequence']}")
            print("Length of first protein sequence:", len(data[first_protein]['sequence']))
else:
    print("Warning: JSON file not found. Preprocessing may have failed.")

# 2. Sampling

In [ ]:
"""
Iterate through all reagent combinations and populate the sample/data folder. Output layout:
sample/data/<proteome_number>/grid_<anchor>_<digest>_<mode>[_capXX]/
"""

from __future__ import annotations
import os
import sys
import subprocess
from itertools import product
from pathlib import Path

# --- Paths ---
PROJECT_ROOT = Path.cwd().resolve()
SAMPLE_DIR   = PROJECT_ROOT / "sample"
RUNNER_MOD   = "sample.GridSampling"

# ------------ Modifiable ---------
proteome_number = "243273"
proteome_json = Path(f"preprocessing/proteomes/protein-data-{proteome_number}.json")
if proteome_number == "243273":
    params_json   = Path("sample/sample_params_myco.json") # mycoplasma allows N-terminal Edman
else:
    params_json   = Path("sample/sample_params.json") # other proteomes have post-translational modifications
iterations    = 1
seed          = 12345
max_workers   = os.cpu_count()
edman_cap     = 15   # used when mode == 'clamp'
fix_grid      = [0.0, 0.01, 0.05, 0.1, 0.5, 1.0]
anchor_grid   = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
digest_grid   = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
store_fragments = False   # if this is turned on, you can view the generated fragments, but it will take up a lot of disk space
# ---------------------------------

anchor_reagents = ["acx", "epoxide"]
digest_reagents = ["trypsin", "lysc", "prok"] 
edman_modes     = ["clamp", "dynamic"]

# --- Sanity check ---
assert proteome_json.exists(), f"Missing {proteome_json}"
assert params_json.exists(),   f"Missing {params_json}"

proteome_number = str(taxon_ID)
OUT_BASE = PROJECT_ROOT / "sample" / "data" / proteome_number
OUT_BASE.mkdir(parents=True, exist_ok=True)

def dir_has_content(p: Path) -> bool:
    """Return True if directory exists and has any files/subdirs."""
    return p.exists() and any(p.iterdir())

def make_out_dir(anchor: str, digest: str, mode: str, cap: int | None = None) -> Path:
    # grid_<anchor>_<digest>_<mode>[_capXX]
    tail = f"grid_{anchor}_{digest}_{mode}"
    if mode == "clamp":
        tail += f"{cap or edman_cap}"
    return OUT_BASE / tail

def run_one(anchor: str, digest: str, mode: str, out_dir: Path) -> None:
    fix_grid_str     = ",".join(f"{x:.2f}" for x in fix_grid)
    anchor_grid_str  = ",".join(f"{x:.2f}" for x in anchor_grid)
    digest_grid_str  = ",".join(f"{x:.2f}" for x in digest_grid)
    cmd = [
        sys.executable, "-m", RUNNER_MOD,
        "--proteome_json",  str(proteome_json.resolve()),
        "--params_json",    str(params_json.resolve()),
        "--out_dir",        str(out_dir.resolve()),
        "--anchor_reagent", anchor,
        "--digest_reagent", digest,
        "--iterations",     str(iterations),
        "--edman_mode",     mode,
        "--edman_cap",      str(edman_cap),
        "--seed",           str(seed),
        "--max_workers",    str(max_workers),
        "--fix_grid",       fix_grid_str,
        "--anch_grid",      anchor_grid_str,
        "--dig_grid",       digest_grid_str,
    ]

    # add the store_fragments flag only if True
    if store_fragments:
        cmd.append("--store_fragments")

    print(f"\n=== Running anchor={anchor} digest={digest} mode={mode} ===")
    print(" ".join(cmd))
    subprocess.run(cmd, check=True, cwd=PROJECT_ROOT)

launched = []
for anchor, digest, mode in product(anchor_reagents, digest_reagents, edman_modes):
    out_dir = make_out_dir(anchor, digest, mode)
    if dir_has_content(out_dir):
        print(f"[skip] {out_dir} already exists and is non-empty; moving to next condition.")
        continue
    out_dir.mkdir(parents=True, exist_ok=True)
    run_one(anchor, digest, mode, out_dir)
    launched.append(out_dir)

print("\nCompleted runs in:")
for p in launched:
    print(" -", p)

# 3. Generating Reference Fragment Dataset

In [ ]:
import json

# ------------ Modifiable ---------
taxon_ID = 243273
# set sampling parameters, which we will store in a json file
reference_params = {
    "sample_depth": 1,
    "test_depth": 1,
    "fixation_prob": {"K": 0.05, "C": 0.05, "Y": 0.05, "R": 0.05, "N_term": 0.05},
    "anchor_prob": {"N_term": 0.8, "K": 0.8}, # AcX anchoring - see GridSampling.py for other options
    "cleave_before_prob": {},
    "cleave_after_prob": {"K": 0.8, "R": 0.8}, # trypsin digestion
    "first_fragment_can_edman_prob": 1, # mycoplasma specific is 1 - for human set this to 0
    "edman_rounds": 15,
    "edman_conjug_fail_prob": 0,
    "edman_cleave_fail_prob": 0,
    "binder_list": ["correct","incorrect"], # 20 aas already accounted for in SampleQueue.py
    "binder_prob": [[1, 0],[0, 1]] 
}
# ---------------------------------

# write parameters to json file
with open('samplereference/reference_params.json', 'w') as f:
    json.dump(reference_params, f)

# get protein sequences
proteome = f'preprocessing/proteomes/protein-data-{taxon_ID}.json'

In [ ]:
'''
Load proteome data and sampling parameters and generate true fragment, error-free dataset.
'''

from samplereference import Sample
import os
import subprocess

# ------------ Modifiable ---------
# set parameters to pass to Sample.py
sample_size = str(reference_params["sample_depth"])
edman_rounds_list = ["5r", "6r", "7r", "8r", "9r", "10r", "11r", "12r", "13r", "14r", "15r"]  # add more rounds as needed
output_dir = f"samplereference/data/{taxon_ID}"
param_path = "samplereference/reference_params.json"
proteome_path = f"preprocessing/proteomes/protein-data-{taxon_ID}.json"
# ---------------------------------

# create output directory
os.makedirs(output_dir, exist_ok=True)

print(f"Checking for existing sampling results...")
print(f"Sample size: {sample_size}")
print(f"Output directory: {output_dir}")
print(f"Edman rounds to process: {edman_rounds_list}")

# loop through different Edman rounds
for edman_rounds in edman_rounds_list:
    print(f"{'='*60}")
    print(f"Processing Edman rounds: {edman_rounds}")
    print(f"{'='*60}")
    
    # check if output files already exist for this round
    pkl_file = os.path.join(output_dir, f"{edman_rounds}{sample_size}.pkl")
    csv_file = os.path.join(output_dir, f"{edman_rounds}{sample_size}.csv")
    
    print(f"Checking for existing files:")
    print(f"  PKL file: {pkl_file} - {'EXISTS' if os.path.exists(pkl_file) else 'MISSING'}")
    print(f"  CSV file: {csv_file} - {'EXISTS' if os.path.exists(csv_file) else 'MISSING'}")
    
    # only run sampling if files don't exist
    if os.path.exists(pkl_file) and os.path.exists(csv_file):
        print(f"Files already exist for {edman_rounds}. Skipping sampling.")
        
        # show file sizes for verification
        pkl_size = os.path.getsize(pkl_file) / (1024 * 1024)  # Convert to MB
        csv_size = os.path.getsize(csv_file) / (1024 * 1024)  # Convert to MB
        print(f"  PKL file size: {pkl_size:.1f} MB")
        print(f"  CSV file size: {csv_size:.1f} MB")
        
    else:
        print(f"Running sampling for {edman_rounds}...")
        
        cmd = [
            "python", "samplereference/Sample.py",
            "--sample_size", sample_size,
            "--round", edman_rounds,
            "--output", output_dir,
            "--param_path", param_path,
            "--proteome_path", proteome_path
        ]

        print("Running Sample.py with command:")
        print(" ".join(cmd))
        print()

        try:
            result = subprocess.run(cmd, capture_output=True, text=True, check=True)
            print("STDOUT:")
            print(result.stdout)
            if result.stderr:
                print("STDERR:")
                print(result.stderr)
            print(f"Sample.py completed successfully for {edman_rounds}!")
            
            # Verify files were created
            if os.path.exists(pkl_file) and os.path.exists(csv_file):
                pkl_size = os.path.getsize(pkl_file) / (1024 * 1024)
                csv_size = os.path.getsize(csv_file) / (1024 * 1024)
                print(f"Files created successfully:")
                print(f"  PKL file size: {pkl_size:.1f} MB")
                print(f"  CSV file size: {csv_size:.1f} MB")
            else:
                print(f"Warning: Expected output files not found for {edman_rounds}")
                
        except subprocess.CalledProcessError as e:
            print(f"Error running Sample.py for {edman_rounds}: {e}")
            print("STDOUT:")
            print(e.stdout)
            print("STDERR:")
            print(e.stderr)
            print(f"Skipping {edman_rounds} and continuing with next round...")
            continue
    
    print()  # Add blank line between rounds

print(f"{'='*60}")
print("All Edman rounds processing completed!")
print(f"{'='*60}")

# Final summary
print(f"\nFinal Summary:")
total_processed = 0
total_skipped = 0

for edman_rounds in edman_rounds_list:
    pkl_file = os.path.join(output_dir, f"{edman_rounds}{sample_size}.pkl")
    csv_file = os.path.join(output_dir, f"{edman_rounds}{sample_size}.csv")
    
    if os.path.exists(pkl_file) and os.path.exists(csv_file):
        total_processed += 1
        print(f"  ✓ {edman_rounds}: Files available")
    else:
        total_skipped += 1
        print(f"  ✗ {edman_rounds}: Files missing")

print(f"\nTotal rounds with files: {total_processed}")
print(f"Total rounds missing files: {total_skipped}")